# LangGraph: Subgraphs and Configuration

## 1. Subgraphs
Subgraphs allow you to nest one `StateGraph` inside another. This is essential for building complex systems where different teams might own different parts of the agentic workflow.

## 2. Configuration
The `config` object allows you to pass logic that isn't part of the core state (like user preferences, API keys, or feature flags).

In [ ]:
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

# --- CHILD GRAPH ---
class ChildState(TypedDict):
    messages: Annotated[list, add_messages]

def child_node(state: ChildState):
    return {"messages": [("ai", "Hello from the Subgraph!")]}

child_builder = StateGraph(ChildState)
child_builder.add_node("child_work", child_node)
child_builder.add_edge(START, "child_work")
child_builder.add_edge("child_work", END)
child_graph = child_builder.compile()


# --- PARENT GRAPH ---
class ParentState(TypedDict):
    messages: Annotated[list, add_messages]

def parent_node(state: ParentState, config: dict):
    # Accessing configuration
    user_type = config.get("configurable", {}).get("user_type", "guest")
    return {"messages": [("ai", f"Parent node acting for a {user_type} user.")]}

parent_builder = StateGraph(ParentState)
parent_builder.add_node("parent_logic", parent_node)

# Nesting the subgraph as a node!
parent_builder.add_node("subgraph_node", child_graph)

parent_builder.add_edge(START, "parent_logic")
parent_builder.add_edge("parent_logic", "subgraph_node")
parent_builder.add_edge("subgraph_node", END)

main_graph = parent_builder.compile()

## 3. Running with Configuration
We pass different configurations to change the behavior of the `parent_node` without changing the state.

In [ ]:
config_premium = {"configurable": {"user_type": "premium"}}
config_guest = {"configurable": {"user_type": "guest"}}

print("--- PREMIUM RUN ---")
for event in main_graph.stream({"messages": [("user", "Go!")]}, config_premium):
    print(event)

print("\n--- GUEST RUN ---")
for event in main_graph.stream({"messages": [("user", "Go!")]}, config_guest):
    print(event)

## Summary
1.  **Subgraphs**: Just treat a compiled `graph` as a node in another graph.
2.  **Configurable**: Use the `config` second argument in your nodes to access out-of-band data (like `thread_id`, `user_id`, or `metadata`).